In [2]:
using JLD2, FileIO

┌ Info: Recompiling stale cache file /Users/manvithaponnapati/.julia/compiled/v1.2/JLD2/O1EyT.ji for JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
└ @ Base loading.jl:1240


In [24]:
f = jldopen("../data/mixed_layer_simulation_Q-100_dTdz0.010_tau0.00_profiles.jld2","r+")
timeseries_keys = keys(f["timeseries"])
print(timeseries_keys)


print(f["timeseries"]["w"]["kappaT"])
for k in keys(f["timeseries"])
#     println(f["timeseries"][k])
end

["t", "vv", "w", "nu", "kappaT", "v", "uv", "ww", "u", "uw", "T", "wT", "uu", "vw"]

KeyError: KeyError: key "kappaT" not found

In [23]:
using DifferentialEquations
function lotka_volterra(du,u,p,t)
  , y = u
  α, β, δ, γ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = -δ*y + γ*x*y
end
u0 = [1.0,1.0]
tspan = (0.0,10.0)
p = [1.5,1.0,3.0,1.0]
prob = ODEProblem(lotka_volterra,u0,tspan,p)
sol = solve(prob,Tsit5())


┌ Info: Recompiling stale cache file /Users/manvithaponnapati/.julia/compiled/v1.2/DifferentialEquations/UQdwS.ji for DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1240


retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 34-element Array{Float64,1}:
  0.0                
  0.0776084743154256 
  0.23264513699277584
  0.4291185174543143 
  0.6790821776882875 
  0.9444045910389707 
  1.2674601253261835 
  1.6192913723304114 
  1.9869755337814992 
  2.264090367186479  
  2.5125486490278424 
  2.746828248824617  
  3.038006827022277  
  ⋮                  
  6.455761908474447  
  6.780495751224447  
  7.171039674548505  
  7.584862904164952  
  7.978068388305894  
  8.483164907244102  
  8.719247868929038  
  8.949206527971544  
  9.200184813643565  
  9.438028630962807  
  9.711807852444823  
 10.0                
u: 34-element Array{Array{Float64,1},1}:
 [1.0, 1.0]                               
 [1.0454942346944578, 0.8576684823217127] 
 [1.1758715885138267, 0.639459570317544]  
 [1.4196809607170826, 0.4569962601282084] 
 [1.8767193485546056, 0.32473343696185236]
 [2.5882499852859384, 0.26336255804531]   
 [3.860708771268753, 0

In [16]:
import Pkg; Pkg.add("DifferentialEquations")



  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed Calculus ──────────────────── v0.5.0
 Installed DiffRules ─────────────────── v0.0.10
 Installed DiffEqJump ────────────────── v6.2.2
 Installed Parameters ────────────────── v0.12.0
 Installed DifferentialEquations ─────── v6.8.0
 Installed ArnoldiMethod ─────────────── v0.0.4
 Installed DelayDiffEq ───────────────── v5.17.0
 Installed DiffEqBase ────────────────── v6.4.1
 Installed PoissonRandom ─────────────── v0.4.0
 Installed ForwardDiff ───────────────── v0.10.4
 Installed MultiScaleArrays ──────────── v1.5.0
 Installed RandomNumbers ─────────────── v1.3.0
 Installed MatrixFactorizations ──────── v0.1.0
 Installed GenericSVD ────────────────── v0.2.2
 Installed Sundials ──────────────────── v3.7.0
 Installed IterativeSolvers ──────────── v0.8.1
 Installed CommonSubexpressions ──────── v0.2.0
 Installed LineSearches ───

In [ ]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

#X -> mxn fl
#Y -> Prediction value

y_dim = 1

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, y_dim),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)